In [2]:
from p2pfl.node import Node
from p2pfl.learning.pytorch.datamodules.mnist import MnistFederatedDM
from p2pfl.learning.pytorch.models.mlp import MLP
from collections import OrderedDict
import torch
import time


def test_convergence(x):
    n,r = x

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r,epochs=0)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.1) #Esperar por la asincronía

        
test_convergence((2,2))

Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 60430
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 60431
INFO:root:Conexión aceptada con ('127.0.0.1', 60431)
INFO:root:Broadcasting start learning...
INFO:root:Training...
INFO:root:Training...
INFO:root:Model added (1/2)
INFO:root:Broadcasting model to all clients...
INFO:root:Model added (1/2)
INFO:root:Broadcasting model to all clients...
INFO:root:Model added (2/2)
INFO:root:Agregating models.
INFO:root:Model added (2/2)
INFO:root:Round 1 of 2 finished. (('127.0.0.1', 60430))
INFO:root:Training...
INFO:root:Model added (1/2)
INFO:root:Broadcasting model to all clients...
INFO:root:Agregating models.
INFO:root:Round 1 of 2 finished. (('127.0.0.1', 60431))
INFO:root:Training...
INFO:root:Model added (1/2)
INFO:root:Broadcasting model to all clients...
INFO:root:Model added (2/2)
INFO:root:Agregating models.
INFO:root:Round 2 of 2 finished. (('127.0.0.1', 60431))
INFO:root:Finish!!.